In [18]:
!which python


/home/pavan/Ds/pro/smart-meeting-copilot/smartcopilot-api/.venv/bin/python


In [1]:

import os
import time
import json
from typing import List, Optional, Dict, TypedDict, Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool

from dotenv import load_dotenv
load_dotenv()

True

In [2]:

groq_api_key = os.getenv("GROQ_API_KEY") 

os.environ["GROQ_API_KEY"] = groq_api_key 

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192") 


In [3]:
llm.invoke("hii")

AIMessage(content="Hii! How's it going?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 12, 'total_tokens': 21, 'completion_time': 0.045381642, 'prompt_time': 0.000246218, 'queue_time': 0.277750322, 'total_time': 0.04562786}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_de9fa14705', 'finish_reason': 'stop', 'logprobs': None}, id='run--05acf5a0-2477-4330-a554-20aac89a5257-0', usage_metadata={'input_tokens': 12, 'output_tokens': 9, 'total_tokens': 21})

In [4]:
# Define State and Structured Models

class ActionItem(BaseModel):
    task: str = Field(description="The specific action or task to be completed.")
    owner: str = Field(description="The person or group responsible for the task.")
    deadline: Optional[str] = Field(None, description="The deadline for the task if mentioned.")
    priority: str = Field("Normal", description="The priority of the task (High, Normal, Low).")


In [5]:
class AgentState(TypedDict):
    transcript_id: str
    summary_id: Optional[str]
    insights_id: Optional[str]
    action_items_id: Optional[str]
    email_id: Optional[str]
    status: Dict[str, Literal["pending", "processing", "success", "error", "retrying"]]
    last_reasoning: str
    error_log: List[str]
    retry_count: Dict[str, int]
    current_step: Literal["supervisor", "summarization", "insights", "action_items", "verification", "slack", "END"]

In [6]:
# Storage implemention (Prototype)

class DataStorage:
    _storage = {
        'transcripts': {},
        'summaries': {},
        'insights': {},
        'action_items': {},
        'emails': {}
    }
    
    @classmethod
    def store(cls, data_type: str, data: any) -> str:
        """Store data and return reference ID"""
        uid = f"{data_type}_{int(time.time())}"
        cls._storage[data_type][uid] = data
        return uid
    
    @classmethod
    def retrieve(cls, data_type: str, uid: str) -> any:
        """Retrieve data by reference ID"""
        return cls._storage[data_type].get(uid)
    

In [7]:
import sys
import os

# Add src directory to Python path
src_dir = os.path.abspath(os.path.join(os.getcwd(), "src"))
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

print(f"Added to path: {src_dir}")

Added to path: /home/pavan/Ds/pro/smart-meeting-copilot/smartcopilot-api/src/notebook/src


In [8]:
# Cell 3.5: MCP Integration via Singleton Manager
import time
import json
import threading
from queue import Queue, Empty
import os

class MCPManager:
    """Singleton MCP Manager for notebook integration with external MCP server"""
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
                    cls._instance._initialized = False
        return cls._instance
    
    def __init__(self):
        if not self._initialized:
            self.mcp_client = None
            self.debug = True
            self._initialized = True
    
    def _get_mcp_client(self):
        """Lazy initialization of MCP client"""
        if self.mcp_client is None:
            try:
                # Import the MCP client from your separate file
                os.chdir('../')
                print(os.getcwd)
                from src.mcpserver import MCPClient  
                self.mcp_client = MCPClient()
                print("📦 MCP Client imported successfully")
            except ImportError as e:
                print(f"❌ Failed to import MCPClient: {e}")
                print("   Make sure mcpserver.py is in your Python path")
                return None
        return self.mcp_client
    
    def start_server(self):
        """Start MCP server and initialize connection"""
        client = self._get_mcp_client()
        if not client:
            return False
        
        try:
            # Start the server process
            if not client.start_server():
                print("❌ Failed to start MCP server")
                return False
            
            # Initialize the connection
            if not client.initialize():
                print("❌ Failed to initialize MCP connection")
                return False
            
            print("✅ MCP server started and initialized successfully")
            return True
            
        except Exception as e:
            print(f"❌ Error starting MCP server: {e}")
            return False
    
    def is_server_running(self):
        """Check if MCP server is running"""
        client = self._get_mcp_client()
        if not client:
            return False
        return hasattr(client, 'running') and client.running and client.initialized
    
    def list_tools(self):
        """List available MCP tools"""
        if not self.is_server_running():
            print("⚠️  Server not running, starting...")
            if not self.start_server():
                return {"error": "Failed to start MCP server"}
        
        client = self._get_mcp_client()
        if not client:
            return {"error": "MCP client not available"}
        
        try:
            result = client.list_tools()
            if self.debug:
                print(f"[TOOLS LIST] {json.dumps(result, indent=2)}")
            return result
        except Exception as e:
            return {"error": f"Failed to list tools: {str(e)}"}
    
    def call_tool(self, tool_name: str, arguments: dict = None) -> dict:
        """Call a specific MCP tool"""
        if not self.is_server_running():
            print("⚠️  Server not running, starting...")
            if not self.start_server():
                return {"error": "Failed to start MCP server"}
        
        client = self._get_mcp_client()
        if not client:
            return {"error": "MCP client not available"}
        
        try:
            result = client.call_tool(tool_name, arguments or {})
            if self.debug:
                print(f"[TOOL CALL] {tool_name} -> {json.dumps(result, indent=2)}")
            return result
        except Exception as e:
            return {"error": f"Failed to call tool {tool_name}: {str(e)}"}
    
    def mcp_request(self, tool_name: str, args: dict = None) -> dict:
        """Alias for call_tool to match your original interface"""
        return self.call_tool(tool_name, args)
    
    def get_server_status(self):
        """Get comprehensive server status"""
        client = self._get_mcp_client()
        if not client:
            return {"status": "client_not_available", "running": False}
        
        status = {
            "running": self.is_server_running(),
            "client_available": True,
            "process_id": getattr(client.process, 'pid', None) if hasattr(client, 'process') and client.process else None
        }
        
        if status["running"]:
            # Try to get tools to verify connectivity
            try:
                tools_response = self.list_tools()
                if (
                    "result" in tools_response
                    and isinstance(tools_response["result"], dict)
                    and "tools" in tools_response["result"]
                    and isinstance(tools_response["result"]["tools"], list)
                ):
                    status["tools_count"] = len(tools_response["result"]["tools"])
                    status["status"] = "healthy"
                    status["tools"] = [t["name"] for t in tools_response["result"]["tools"]]
                else:
                    status["status"] = "connection_issues"
                    status["error"] = tools_response.get("error", "Unknown error")
            except Exception as e:
                status["status"] = "connection_error"
                status["error"] = str(e)
        else:
            status["status"] = "not_running"
        
        return status
    
    def restart_server(self):
        """Restart the MCP server"""
        print("🔄 Restarting MCP server...")
        self.stop_server()
        time.sleep(2)
        return self.start_server()
    
    def stop_server(self):
        """Stop the MCP server"""
        client = self._get_mcp_client()
        if client and hasattr(client, 'stop'):
            try:
                client.stop()
                print("✅ MCP server stopped")
            except Exception as e:
                print(f"⚠️  Error stopping server: {e}")
    
    def test_connection(self):
        """Test MCP connection with basic tool listing"""
        print("🧪 Testing MCP connection...")
        
        status = self.get_server_status()
        print(f"📊 Server Status: {status['status']}")
        
        if status["status"] == "healthy":
            tools = status.get("tools", [])
            print(f"✅ Connection successful! Found {len(tools)} tools:")
            for i, tool_name in enumerate(tools[:5], 1):  # Show first 5 tools
                print(f"  {i}. {tool_name}")
            if len(tools) > 5:
                print(f"  ... and {len(tools) - 5} more tools")
            return True
        else:
            print(f"❌ Connection failed: {status.get('error', 'Unknown error')}")
            return False
    
    def show_tools(self):
        """Display available tools in a user-friendly format"""
        tools_response = self.list_tools()
        
        if "error" in tools_response:
            print(f"❌ Error getting tools: {tools_response['error']}")
            return
        
        if "result" not in tools_response:
            print("❌ No tools found or invalid response")
            return

        result = tools_response["result"]
        # Handle both dict and list cases for result
        if isinstance(result, dict) and "tools" in result and isinstance(result["tools"], list):
            tools = result["tools"]
        elif isinstance(result, list):
            tools = result
        else:
            print("❌ No tools found or invalid response structure")
            return

        print(f"\n📋 Available Tools ({len(tools)}):")
        print("=" * 50)
        
        for i, tool in enumerate(tools, 1):
            print(f"{i}. 🔧 {tool.get('name', 'Unnamed Tool')}")
            if 'description' in tool:
                print(f"   📝 {tool['description']}")
            if 'inputSchema' in tool and isinstance(tool['inputSchema'], dict) and 'properties' in tool['inputSchema']:
                props = tool['inputSchema']['properties']
                if props:
                    print(f"   📥 Parameters: {', '.join(props.keys())}")
            print()
    
    def quick_call(self, tool_name: str, **kwargs):
        """Quick tool call with keyword arguments"""
        return self.call_tool(tool_name, kwargs)

# Initialize MCP manager (singleton)
print("🔧 Initializing MCP Manager...")
mcp_manager = MCPManager()

# Test if we can import the MCP client
try:
    client_test = mcp_manager._get_mcp_client()
    if client_test:
        print("✅ MCP Client import successful")
        
        # Auto-start server
        print("🚀 Starting MCP server...")
        if mcp_manager.start_server():
            print("🎉 MCP Manager ready for use!")
            
            # Quick connection test
            if mcp_manager.test_connection():
                print("\n📋 Use mcp_manager.show_tools() to see all available tools")
        else:
            print("⚠️  MCP server failed to start. Use mcp_manager.start_server() to retry")
    else:
        print("⚠️  MCP Client not available. Make sure mcpserver.py is accessible")
        
except Exception as e:
    print(f"⚠️  MCP initialization error: {e}")
    print("   The manager is available but server needs manual start")


🔧 Initializing MCP Manager...
<built-in function getcwd>
❌ Failed to import MCPClient: No module named 'src'
   Make sure mcpserver.py is in your Python path
⚠️  MCP Client not available. Make sure mcpserver.py is accessible


In [9]:
mcp_manager.call_tool(tool_name= "slack_post_message", arguments= {
    "channel_id": 'C0934AS6UB1', 
    "text": "testing no.3"
})

<built-in function getcwd>
📦 MCP Client imported successfully
⚠️  Server not running, starting...
⏳ Waiting for MCP server to start...
[MCP STDERR] Starting Slack MCP Server...
[MCP STDERR] Connecting server to transport...
[MCP STDERR] Slack MCP Server running on stdio
✅ MCP Server started successfully

🤝 Sending initialize...
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {},
    "clientInfo": {
      "name": "mcp-python-client",
      "version": "1.0"
    }
  },
  "id": 1
}
[MCP STDOUT] {"result":{"protocolVersion":"2024-11-05","capabilities":{"tools":{}},"serverInfo":{"name":"Slack MCP Server","version":"1.0.0"}},"jsonrpc":"2.0","id":1}
[MCP RESPONSE] {
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {}
    },
    "serverInfo": {
      "name": "Slack MCP Server",
      "version": "1.0.0"
    }
  },
  "jsonrpc": "2.0",
  "id": 1
}
✅ Initialize successful
📤 Se

{'result': {'content': [{'type': 'text',
    'text': '{"ok":true,"channel":"C0934AS6UB1","ts":"1752753902.610729","message":{"user":"U0946J1TQHW","type":"message","ts":"1752753902.610729","bot_id":"B0946J1T4M6","app_id":"A093GGX8TE1","text":"testing no.3","team":"T093JHQS18A","bot_profile":{"id":"B0946J1T4M6","app_id":"A093GGX8TE1","user_id":"U0946J1TQHW","name":"meetingplus","icons":{"image_36":"https://a.slack-edge.com/80588/img/plugins/app/bot_36.png","image_48":"https://a.slack-edge.com/80588/img/plugins/app/bot_48.png","image_72":"https://a.slack-edge.com/80588/img/plugins/app/service_72.png"},"deleted":false,"updated":1751170336,"team_id":"T093JHQS18A"},"blocks":[{"type":"rich_text","block_id":"KUsy","elements":[{"type":"rich_text_section","elements":[{"type":"text","text":"testing no.3"}]}]}]},"warning":"missing_charset","response_metadata":{"warnings":["missing_charset"]}}'}]},
 'jsonrpc': '2.0',
 'id': 2}

[MCP STDERR] Received CallToolRequest: {
[MCP STDERR] method: 'tools/call',
[MCP STDERR] params: { name: 'slack_list_channels', arguments: {} }
[MCP STDERR] }
[MCP STDOUT] {"result":{"content":[{"type":"text","text":"{\"ok\":true,\"channels\":[{\"id\":\"C093KK8DBGC\",\"name\":\"ai-team\",\"is_channel\":true,\"is_group\":false,\"is_im\":false,\"is_mpim\":false,\"is_private\":false,\"created\":1751170632,\"is_archived\":false,\"is_general\":false,\"unlinked\":0,\"name_normalized\":\"ai-team\",\"is_shared\":false,\"is_org_shared\":false,\"is_pending_ext_shared\":false,\"pending_shared\":[],\"context_team_id\":\"T093JHQS18A\",\"updated\":1751170632496,\"parent_conversation\":null,\"creator\":\"U093JHQT2JW\",\"is_ext_shared\":false,\"shared_team_ids\":[\"T093JHQS18A\"],\"pending_connected_team_ids\":[],\"is_member\":true,\"last_read\":\"1751276461.859299\",\"topic\":{\"value\":\"\",\"creator\":\"\",\"last_set\":0},\"purpose\":{\"value\":\"\",\"creator\":\"\",\"last_set\":0},\"previous_names

In [10]:
mcp_manager.call_tool(tool_name= "slack_list_channels", arguments= {})

[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_list_channels",
    "arguments": {}
  },
  "id": 3
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"channels\":[{\"id\":\"C093KK8DBGC\",\"name\":\"ai-team\",\"is_channel\":true,\"is_group\":false,\"is_im\":false,\"is_mpim\":false,\"is_private\":false,\"created\":1751170632,\"is_archived\":false,\"is_general\":false,\"unlinked\":0,\"name_normalized\":\"ai-team\",\"is_shared\":false,\"is_org_shared\":false,\"is_pending_ext_shared\":false,\"pending_shared\":[],\"context_team_id\":\"T093JHQS18A\",\"updated\":1751170632496,\"parent_conversation\":null,\"creator\":\"U093JHQT2JW\",\"is_ext_shared\":false,\"shared_team_ids\":[\"T093JHQS18A\"],\"pending_connected_team_ids\":[],\"is_member\":true,\"last_read\":\"1751276461.859299\",\"topic\":{\"value\":\"\",\"creator\":\"\",\"last_set\":0},\"purpose\":{\"value\":\"\",\"creator\":\"\",\"la

{'result': {'content': [{'type': 'text',
    'text': '{"ok":true,"channels":[{"id":"C093KK8DBGC","name":"ai-team","is_channel":true,"is_group":false,"is_im":false,"is_mpim":false,"is_private":false,"created":1751170632,"is_archived":false,"is_general":false,"unlinked":0,"name_normalized":"ai-team","is_shared":false,"is_org_shared":false,"is_pending_ext_shared":false,"pending_shared":[],"context_team_id":"T093JHQS18A","updated":1751170632496,"parent_conversation":null,"creator":"U093JHQT2JW","is_ext_shared":false,"shared_team_ids":["T093JHQS18A"],"pending_connected_team_ids":[],"is_member":true,"last_read":"1751276461.859299","topic":{"value":"","creator":"","last_set":0},"purpose":{"value":"","creator":"","last_set":0},"previous_names":[]},{"id":"C093XNMPFLZ","name":"designing-team","is_channel":true,"is_group":false,"is_im":false,"is_mpim":false,"is_private":false,"created":1751170654,"is_archived":false,"is_general":false,"unlinked":0,"name_normalized":"designing-team","is_shared":fa

In [14]:
args = {"limit": 2}
 
res = mcp_manager.mcp_request( tool_name ="slack_get_users", args= args)


[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_get_users",
    "arguments": {
      "limit": 2
    }
  },
  "id": 4
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"offset\":\"U0946HH4GNL\",\"members\":[{\"id\":\"USLACKBOT\",\"name\":\"slackbot\",\"is_bot\":false,\"updated\":0,\"is_app_user\":false,\"team_id\":\"T093JHQS18A\",\"deleted\":false,\"color\":\"757575\",\"is_email_confirmed\":false,\"real_name\":\"Slackbot\",\"tz\":\"America/Los_Angeles\",\"tz_label\":\"Pacific Daylight Time\",\"tz_offset\":-25200,\"is_admin\":false,\"is_owner\":false,\"is_primary_owner\":false,\"is_restricted\":false,\"is_ultra_restricted\":false,\"who_can_share_contact_card\":\"EVERYONE\",\"profile\":{\"real_name\":\"Slackbot\",\"display_name\":\"Slackbot\",\"avatar_hash\":\"sv41d8cd98f0\",\"real_name_normalized\":\"Slackbot\",\"display_name_normalized\":\"Slackbot\",\"image_24\":\"https://a.sla

In [15]:

res1 = res.get("result",[])
# res2 = res1.get('text')

res2 = res1.get('content')[0].get('text')

res2

'{"ok":true,"offset":"U0946HH4GNL","members":[{"id":"USLACKBOT","name":"slackbot","is_bot":false,"updated":0,"is_app_user":false,"team_id":"T093JHQS18A","deleted":false,"color":"757575","is_email_confirmed":false,"real_name":"Slackbot","tz":"America/Los_Angeles","tz_label":"Pacific Daylight Time","tz_offset":-25200,"is_admin":false,"is_owner":false,"is_primary_owner":false,"is_restricted":false,"is_ultra_restricted":false,"who_can_share_contact_card":"EVERYONE","profile":{"real_name":"Slackbot","display_name":"Slackbot","avatar_hash":"sv41d8cd98f0","real_name_normalized":"Slackbot","display_name_normalized":"Slackbot","image_24":"https://a.slack-edge.com/80588/img/slackbot_24.png","image_32":"https://a.slack-edge.com/80588/img/slackbot_32.png","image_48":"https://a.slack-edge.com/80588/img/slackbot_48.png","image_72":"https://a.slack-edge.com/80588/img/slackbot_72.png","image_192":"https://a.slack-edge.com/80588/marketing/img/avatars/slackbot/avatar-slackbot.png","image_512":"https://a

In [16]:
res3 = json.loads(res2)

res3.get('members')[1]

{'id': 'U093JHQT2JW',
 'name': 'cme04408',
 'is_bot': False,
 'updated': 1751169830,
 'is_app_user': False,
 'team_id': 'T093JHQS18A',
 'deleted': False,
 'color': '9b3b45',
 'is_email_confirmed': True,
 'real_name': 'Pavan Manikanta',
 'tz': 'Asia/Kolkata',
 'tz_label': 'India Standard Time',
 'tz_offset': 19800,
 'is_admin': True,
 'is_owner': True,
 'is_primary_owner': True,
 'is_restricted': False,
 'is_ultra_restricted': False,
 'who_can_share_contact_card': 'EVERYONE',
 'profile': {'real_name': 'Pavan Manikanta',
  'display_name': '',
  'avatar_hash': '6d3936e1f916',
  'real_name_normalized': 'Pavan Manikanta',
  'display_name_normalized': '',
  'image_24': 'https://avatars.slack-edge.com/2025-06-28/9142579702096_6d3936e1f916a570271f_24.jpg',
  'image_32': 'https://avatars.slack-edge.com/2025-06-28/9142579702096_6d3936e1f916a570271f_32.jpg',
  'image_48': 'https://avatars.slack-edge.com/2025-06-28/9142579702096_6d3936e1f916a570271f_48.jpg',
  'image_72': 'https://avatars.slack-ed

In [11]:
for user in range(len(res3.get('members'))):
    
    man = res3.get('members')[user]

    print(man.get('real_name'))


NameError: name 'res3' is not defined

In [ ]:

name = "Pavan Manislackbotkanta"
result  = mcp_manager.mcp_request( tool_name ="slack_get_users", args= {"limit": 200})
    
result1 = result.get("result", [])

res2 = result1.get('content')[0].get('text')

raw = json.loads(res2)

users = raw.get('members')

for user in users:
    
    # print(user.get("profile").get('real_name'))

    # print(user)
            
    if name.lower() in [
    user.get("name", "").lower(),
    user.get("real_name", "").lower(),
    user.get("profile", "").get('real_name').lower()
    ]:
        print("done:", user)

    else:
        print("not found ")


[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_get_users",
    "arguments": {
      "limit": 200
    }
  },
  "id": 35
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"members\":[{\"id\":\"USLACKBOT\",\"name\":\"slackbot\",\"is_bot\":false,\"updated\":0,\"is_app_user\":false,\"team_id\":\"T093JHQS18A\",\"deleted\":false,\"color\":\"757575\",\"is_email_confirmed\":false,\"real_name\":\"Slackbot\",\"tz\":\"America/Los_Angeles\",\"tz_label\":\"Pacific Daylight Time\",\"tz_offset\":-25200,\"is_admin\":false,\"is_owner\":false,\"is_primary_owner\":false,\"is_restricted\":false,\"is_ultra_restricted\":false,\"who_can_share_contact_card\":\"EVERYONE\",\"profile\":{\"real_name\":\"Slackbot\",\"display_name\":\"Slackbot\",\"avatar_hash\":\"sv41d8cd98f0\",\"real_name_normalized\":\"Slackbot\",\"display_name_normalized\":\"Slackbot\",\"image_24\":\"https://a.slack-edge.com/80588/img/sl

In [15]:
res

{'result': {'content': [{'type': 'text',
    'text': '{"ok":true,"offset":"U0946HH4GNL","members":[{"id":"USLACKBOT","name":"slackbot","is_bot":false,"updated":0,"is_app_user":false,"team_id":"T093JHQS18A","deleted":false,"color":"757575","is_email_confirmed":false,"real_name":"Slackbot","tz":"America/Los_Angeles","tz_label":"Pacific Daylight Time","tz_offset":-25200,"is_admin":false,"is_owner":false,"is_primary_owner":false,"is_restricted":false,"is_ultra_restricted":false,"who_can_share_contact_card":"EVERYONE","profile":{"real_name":"Slackbot","display_name":"Slackbot","avatar_hash":"sv41d8cd98f0","real_name_normalized":"Slackbot","display_name_normalized":"Slackbot","image_24":"https://a.slack-edge.com/80588/img/slackbot_24.png","image_32":"https://a.slack-edge.com/80588/img/slackbot_32.png","image_48":"https://a.slack-edge.com/80588/img/slackbot_48.png","image_72":"https://a.slack-edge.com/80588/img/slackbot_72.png","image_192":"https://a.slack-edge.com/80588/marketing/img/avatars

In [ ]:

print("\n📚 Available methods:")
print("  🔧 mcp_manager.show_tools()                    # Show all available tools")
print("  ⚡ mcp_manager.call_tool(name, args)           # Call a specific tool")  
print("  🚀 mcp_manager.quick_call(name, **kwargs)      # Call tool with keyword args")
print("  📊 mcp_manager.get_server_status()             # Check server status")
print("  🧪 mcp_manager.test_connection()               # Test connection")
print("  🔄 mcp_manager.restart_server()                # Restart server")
print("  🛑 mcp_manager.stop_server()                   # Stop server")


In [15]:
import time

def say_hello():
    time.sleep(3)
    print("Hello, Async World? (not yet)")

say_hello()

Hello, Async World? (not yet)


In [1]:
!which python

/home/pavan/Ds/pro/smart-meeting-copilot/smartcopilot-api/.venv/bin/python


In [3]:
!uv pip install asyncio

Using Python 3.12.3 environment at: /home/pavan/Ds/pro/smart-meeting-copilot/smartcopilot-api/.venv
Resolved 1 package in 927ms                                          
⠸ Preparing packages... (0/1)                                                   
⠼ Preparing packages... (0/1)--------------     0 B/99.38 KiB           
⠼ Preparing packages... (0/1)--------------     0 B/99.38 KiB           
⠴ Preparing packages... (0/1)-------------- 30.84 KiB/99.38 KiB         
⠴ Preparing packages... (0/1)-------------- 30.84 KiB/99.38 KiB         
⠴ Preparing packages... (0/1)-------------- 46.62 KiB/99.38 KiB         
⠦ Preparing packages... (0/1)-------------- 48.97 KiB/99.38 KiB         
⠦ Preparing packages... (0/1)-------------- 48.97 KiB/99.38 KiB         
⠦ Preparing packages... (0/1)---------- 64.97 KiB/99.38 KiB         
⠦ Preparing packages... (0/1)---------- 78.84 KiB/99.38 KiB         
⠦ Preparing packages... (0/1)---------- 94.84 KiB/99.38 KiB         
Prepared 1 package in 1.03s    

In [11]:
import asyncio

async def say_hello_async():
    await asyncio.sleep(2)
    print("Hello, Async World!")


asyncio.run(say_hello_async())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [12]:
import asyncio

async def say_hello_async():
    await asyncio.sleep(2)  # Simulates waiting for 2 seconds
    print("Hello, Async World!")

async def do_something_else():
    print("Starting another task...")
    await asyncio.sleep(1)  # Simulates doing something else for 1 second
    print("Finished another task!")

async def main():
    # Schedule both tasks to run concurrently
    await asyncio.gather(
        say_hello_async(),
        do_something_else(),
    )

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
#  Enhanced Tools with MCP Integration via Stdio
@tool
def slack_post_message(channel_id: str, text: str) -> dict:
    """Post message to Slack channel via MCP server"""
    # Truncate to Slack's limits
    if len(text) > 3900:
        text = text[:3900] + "\n\n[Message truncated]"
    
    return mcp_manager.call_tool(
       tool_name = "slack_post_message",
        arguments = {"channel_id": channel_id, "text": text}
    )

@tool
def slack_list_channels(limit: int = 100, cursor: str = None) -> dict:
    """List accessible Slack channels via MCP server"""
    args = {"limit": limit}
    if cursor:
        args["cursor"] = cursor
    return mcp_manager.call_tool(tool_name = "slack_list_channels", arguments = args)

@tool
def slack_get_users(limit: int = 100, cursor: str = None) -> dict:
    """Get workspace users via MCP server"""
    args = {"limit": limit}
    if cursor:
        args["cursor"] = cursor
    return mcp_manager.call_tool( tool_name ="slack_get_users", arguments= args)

@tool
def slack_find_user_by_name(name: str) -> Optional[str]:
    """Find user by display name using MCP tools"""
    try:
        result = slack_get_users(limit=200)
        result1 = result.get("result", [])

        res2 = result1.get('content')[0].get('text')

        raw = json.loads(res2)

        users = raw.get('members')
        
        for user in users:

            print(user)
            
            if name.lower() in [
                    user.get("name", "").lower(),
                    user.get("real_name", "").lower(),
                    user.get("profile", "").get('real_name').lower()
             ]:
                return { "user_id" : user["id"], "user_name":user }
        return None
    except Exception:
        return None

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent

def slack_node( message : str ):

    
    print("*****************slack NODE CALLED************")
    
    system_prompt = """ you are an intelliagent sclak agent. your job is to:
    <Instruction>
    1. use the different tools to get the relate information. then pass message over the slack channels  
    </Instruction>
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    # Create the react agent with tools
    slack_agent = create_react_agent(
        model=llm,
        tools=[slack_find_user_by_name, slack_get_users, slack_post_message, slack_list_channels ],
        prompt=prompt
    )
    
    try:
        
        result = slack_agent.invoke({"messages": message})
        
        print(f"Slack agent result: {result}")

        return result
    
    except Exception as e:
        print(f" Error in slack_node: {e}")


In [22]:
slack_node("send a message ( Good Morning to ALL from Slack agent ) on ai team channel ")

*****************slack NODE CALLED************
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_post_message",
    "arguments": {
      "channel_id": "ai_team_channel",
      "text": "Good Morning to ALL from Slack agent"
    }
  },
  "id": 4
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":false,\"error\":\"channel_not_found\",\"warning\":\"missing_charset\",\"response_metadata\":{\"warnings\":[\"missing_charset\"]}}"
      }
    ]
  },
  "jsonrpc": "2.0",
  "id": 4
}
[TOOL CALL] slack_post_message -> {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":false,\"error\":\"channel_not_found\",\"warning\":\"missing_charset\",\"response_metadata\":{\"warnings\":[\"missing_charset\"]}}"
      }
    ]
  },
  "jsonrpc": "2.0",
  "id": 4
}
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_list_channels",
    "

{'messages': [HumanMessage(content='send a message ( Good Morning to ALL from Slack agent ) on ai team channel ', additional_kwargs={}, response_metadata={}, id='a5946ea7-b844-4043-9c1d-4800d1dd7980'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '6892sqg1c', 'function': {'arguments': '{"channel_id":"ai_team_channel","text":"Good Morning to ALL from Slack agent"}', 'name': 'slack_post_message'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 1342, 'total_tokens': 1401, 'completion_time': 0.179229908, 'prompt_time': 0.050419686, 'queue_time': 0.057482513000000006, 'total_time': 0.229649594}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-14234766-497f-4e52-ba80-218d53682526-0', tool_calls=[{'name': 'slack_post_message', 'args': {'channel_id': 'ai_team_channel', 'text': 'Good Morning to ALL from Slack agent'}, 'id': '6892sqg1c', 'type

In [22]:
ress = slack_node("send a message ( Good Evening by tagging slackbot ) like @pa... on general channel all-abc")

*****************slack NODE CALLED************
[MCP REQUEST] {
  "jsonrpc": "2.0",
  "method": "tools/call",
  "params": {
    "name": "slack_post_message",
    "arguments": {
      "channel_id": "all-abc",
      "text": "Good Evening <@UACLKJQJQ>!"
    }
  },
  "id": 6
}
[MCP RESPONSE] {
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\"ok\":true,\"channel\":\"C093JHR4DNE\",\"ts\":\"1751378627.804079\",\"message\":{\"user\":\"U0946J1TQHW\",\"type\":\"message\",\"ts\":\"1751378627.804079\",\"bot_id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"text\":\"Good Evening <@UACLKJQJQ>!\",\"team\":\"T093JHQS18A\",\"bot_profile\":{\"id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"user_id\":\"U0946J1TQHW\",\"name\":\"meetingplus\",\"icons\":{\"image_36\":\"https://a.slack-edge.com/80588/img/plugins/app/bot_36.png\",\"image_48\":\"https://a.slack-edge.com/80588/img/plugins/app/bot_48.png\",\"image_72\":\"https://a.slack-edge.com/80588/img/plugins/app/service_72

In [23]:
ress

{'messages': [HumanMessage(content='send a message ( Good Evening by tagging slackbot ) like @pa... on general channel all-abc', additional_kwargs={}, response_metadata={}, id='145cf0ab-38f3-4594-9d69-f91e428a50da'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7qk9z8sy1', 'function': {'arguments': '{"channel_id":"all-abc","text":"Good Evening \\u003c@UACLKJQJQ\\u003e!"}', 'name': 'slack_post_message'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 1288, 'total_tokens': 1353, 'completion_time': 0.230101047, 'prompt_time': 0.04961946, 'queue_time': 0.053490289, 'total_time': 0.279720507}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6720ca59-7bad-496d-9778-8649316b4645-0', tool_calls=[{'name': 'slack_post_message', 'args': {'channel_id': 'all-abc', 'text': 'Good Evening <@UACLKJQJQ>!'}, 'id': '7qk9z8sy1', 'type': 'tool_call'}], 

In [24]:
for message in ress["messages"]:
    message.pretty_print()

================================ Human Message =================================

send a message ( Good Evening by tagging slackbot ) like @pa... on general channel all-abc
================================== Ai Message ==================================
Tool Calls:
  slack_post_message (7qk9z8sy1)
 Call ID: 7qk9z8sy1
  Args:
    channel_id: all-abc
    text: Good Evening <@UACLKJQJQ>!
================================= Tool Message =================================
Name: slack_post_message

{"result": {"content": [{"type": "text", "text": "{\"ok\":true,\"channel\":\"C093JHR4DNE\",\"ts\":\"1751378627.804079\",\"message\":{\"user\":\"U0946J1TQHW\",\"type\":\"message\",\"ts\":\"1751378627.804079\",\"bot_id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"text\":\"Good Evening <@UACLKJQJQ>!\",\"team\":\"T093JHQS18A\",\"bot_profile\":{\"id\":\"B0946J1T4M6\",\"app_id\":\"A093GGX8TE1\",\"user_id\":\"U0946J1TQHW\",\"name\":\"meetingplus\",\"icons\":{\"image_36\":\"https://a.slack-edge.com/80588/i